# Cluster SPRM output of CODEX data and visualize with Vitessce
This template shows how to cluster CODEX data and load a new clustering into Vitessce. This template requires prior cell segmentation, and takes in the locations of cell (centers) and protein expression per cell. 


In [ ]:
!pip install numpy pandas matplotlib matplotlib-inline scikit-learn anndata vitessce starlette uvicorn


In [ ]:
# import modules
import os
import json
import requests
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import anndata as ad

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from vitessce import VitessceChainableConfig, AnnDataWrapper

## Linked datasets
The following datasets were symlinked to the workspace when this template was added:

In [ ]:
# linked datasets
uuids = {{ uuids | safe }}

# accepted datatypes 
accepted_datatypes = ['CODEX [Cytokit + SPRM]']

# required filetypes
required_filetypes = ['sprm_outputs/reg001_expr.ome.tiff-cell_channel_total.csv', 'sprm_outputs/reg001_expr.ome.tiff-cell_channel_mean.csv', 'sprm_outputs/reg001_expr.ome.tiff-cell_centers.csv']

# search_api
search_api = 'https://search.api.hubmapconsortium.org/v3/portal/search'

The following checks if the datasets are compatible with this template.

In [ ]:
# This template is created for particular datatypes only.
# This functions checks for each uuids above whether they have the correct datatypes.

def check_template_compatibility(uuids, accepted_datatypes=None, required_filetypes=None, search_api = 'https://search.api.hubmapconsortium.org/v3/portal/search'): 
    '''
    For a set of HuBMAP UUIDs, check if valid, and return valid UUIDs.
    Checks if UUIDs are present in the search API. 
    If accepted_datatypes is defined, checks if UUIDs are of any of the datatypes in accepted_datatypes.
    If required_filetypes is defined, checks if UUIDs have all of the required filetypes in required_filetypes.

    Parameters
    ----------
    uuids : array of string
        HuBMAP UUIDs to be checked
    accepted_datatypes: array of string, optional
        accepted datatypes for template
    required_filetypes: array of string, optional
        required datatypes for template
    search_api: string, optional
        URL of search API

    Returns
    -------
    array of string
        valid UUIDs
    '''
    hits = json.loads(
        requests.post(
            search_api,
            json={
                'size': 10000,
                'query': {'ids': {'values': uuids}},
                '_source': ['files', 'assay_display_name']
            }, 
        ).text
    )['hits']['hits']

    # create mapping for uuid to file_types and assay_display_name
    uuid_to_files = {}
    uuid_to_datatypes = {}
    for hit in hits:
        file_paths = [file['rel_path'] for file in hit['_source']['files']]
        uuid_to_files[hit['_id']] = file_paths

        hit_data_type = hit['_source']['assay_display_name']
        uuid_to_datatypes[hit['_id']] = hit_data_type
    
    # save uuids without warnings
    accepted_uuids = uuids.copy()

    # remove unvalid uuids
    for uuid in uuids: 
        # check if all uuids are found in the search api
        if uuid not in uuid_to_files.keys(): 
            warnings.warn('Dataset with UUID "' + uuid + '" not found in Search API')
            accepted_uuids.remove(uuid)
            continue

        if required_filetypes is not None: 
            # check if file_types for each uuid are in required_filetypes
            file_types = uuid_to_files[uuid]
            for required_file_type in required_filetypes:
                if required_file_type not in file_types:
                    warnings.warn('Dataset with UUID "' + uuid + '" does not have required file type: ' + required_file_type)
                    if uuid in accepted_uuids:
                        accepted_uuids.remove(uuid)

        if accepted_datatypes is not None: 
            # check if assay_display_name for each uuid are in accepted_datatypes
            assay_display_name = uuid_to_datatypes[uuid]
            for data_type in assay_display_name:
                if data_type not in accepted_datatypes: 
                    warnings.warn('Dataset with UUID "' + uuid + '" has unaccepted data type: ' + data_type)
                    if uuid in accepted_uuids:
                        accepted_uuids.remove(uuid)
                    continue
    
    return accepted_uuids

In [ ]:
uuids = check_template_compatibility(uuids, accepted_datatypes=accepted_datatypes, required_filetypes=required_filetypes, search_api=search_api)

## Data retrieval

This template will work with the total and mean expression as determined by the SPRM pipeline, and will use the determined cell centers. If the datasets are symlinked with the workspace, these files are already available locally. Otherwise, below shows how to retrieve these specific files through the assets API. As these are relatively small csv files, this is feasible. However, for many files or larger files, it is faster to symlink the datasets in a workspace.

In [ ]:
def retrieve_sprm_file(uuid, file_name, root_folder='.'): 
    '''
    For a given UUID and SPRM output file name, retrieve this file and save it locally.

    Parameters: 
        uuid (string): UUID of dataset
        file_name (string): desired file
    '''
    if not os.path.exists('datasets/' + uuid + '/sprm_outputs'):
        # unlike os.mkdir, os.makedirs creates directories recursively
        os.makedirs('datasets/' + uuid + '/sprm_outputs', exist_ok = True)

    sprm_url = 'https://assets.hubmapconsortium.org/' + uuid + '/sprm_outputs/' + file_name
    res = requests.get(sprm_url)
    if res.status_code != 200:
        if res.status_code == 404: 
            raise FileNotFoundError('For uuid:', uuid, 'and file', file_name, 'generated URL does not exist.',
                                    'Did you check the file name and whether this is an SPRM dataset? Generated url:', 
                                    sprm_url, 
                                    'File should be visible in', 'https://portal.hubmapconsortium.org/browse/dataset/' + uuid)
        else: 
            raise ConnectionError('Error code:', res.status_code, 
                                  'For uuid:', uuid, 'and file', file_name, '. Did you check the UUID?')
    else:
        with open(root_folder + '/datasets/' + uuid + '/sprm_outputs/' + file_name, 'wb') as f: 
            f.write(res.content)


def retrieve_sprm_files(uuids, file_names=['reg001_expr.ome.tiff-cell_channel_total.csv', 'reg001_expr.ome.tiff-cell_channel_mean.csv', 'reg001_expr.ome.tiff-cell_centers.csv'], root_folder='.'):
    '''
    For a given list of UUIDS, retrieve files for sprm analysis and save these locally in datasets folder.

    Parameters: 
        uuids (list): list of uuids of interest
        file_name (list): desired files. Default: ['reg001_expr.ome.tiff-cell_channel_total.csv', 'reg001_expr.ome.tiff-cell_channel_mean.csv', 'reg001_expr.ome.tiff-cell_centers.csv']
    '''
    for uuid in uuids: 
        for file in file_names: 
            if not (os.path.exists(root_folder + '/datasets/' + uuid + '/sprm_outputs/' + file)): 
                retrieve_sprm_file(uuid, file)

In [ ]:
root_folder = '.'

retrieve_sprm_files(uuids, root_folder=root_folder)

## K-means clustering
This shows how to apply k-means clustering to a particular dataframe, with variable amount of clusters and optional scalers. 

In [ ]:
def kmeans_pipe(df, n_clusters=10, scaler='z'):
    '''
    Pipeline for applying preprocessing and k-means clustering to dataset.

    Parameters
    ----------
    df : pandas DataFrame
        dataframe with protein expressions per cell
    n_clusters : int, optional
        number of clusters for k-means clustering. Default: 10
    scaler : str, optional
        Scaler used for preprocessing. One of 'z', 'minmax' or None. Default: 'z'

    Returns
    -------
    pandas DataFrame
        Result of k-means clustering. Dataframe with 2 columns (ID = cell ID, cluster = cluster label)
    '''
    if scaler == 'z':
        preprocessor = Pipeline(
            [
                ("scaler", StandardScaler()),
                ("pca", PCA(n_components=2, random_state=42)),
            ]
        )
    elif scaler == 'minmax': 
        preprocessor = Pipeline(
            [
                ("scaler", MinMaxScaler()),
                ("pca", PCA(n_components=2, random_state=42)),
            ]
            )
    elif scaler == None:
        preprocessor = Pipeline(
            [
                ("pca", PCA(n_components=2, random_state=42)),
            ]
            )
        
    else:
        warnings.warn(scaler + " is not a valid scaling option. Defaulting to z-scaler.")
        preprocessor = Pipeline(
            [
                ("scaler", StandardScaler()),
                ("pca", PCA(n_components=2, random_state=42)),
            ]
            )
    
    clusterer = Pipeline(
        [
         (
           "kmeans",
           KMeans(
               n_clusters=n_clusters,
               init="k-means++",
               n_init=50,
               max_iter=500,
               random_state=42,
           ),
       ),
        ]
    )
    pipe = Pipeline(
     [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
        ]
    )
    pipe.fit(df)

    kmdf = pd.DataFrame()

    kmdf['ID'] = df['ID']
    kmdf["cluster"] = pipe["clusterer"]["kmeans"].labels_
    
    return kmdf

We might also want to subset our proteins.

In [ ]:
#funciton for getting all proteins
def get_all_proteins(df):
    '''
    Return a list of all proteins in dataset.

    Parameters
    ----------
    df : pandas DataFrame
        dataframe with protein expressions per cell

    Returns
    -------
    list of string
        list with names of proteins
    '''
    return df.columns

#function for creating a subset of a dataframe using a list of columns
def protein_subset(df, protein_list, keep=True):
    '''
    Subset dataframe for certain proteins. 
    If keep=True, keep the proteins in protein_list. 
    Otherwise, keep all proteins not in protein_list.

    Parameters
    ----------
    df : pandas DataFrame
        dataframe with protein expressions per cell
    protein_list : list of string
        list with proteins to keep or discard
    keep : Boolean, optional
        whether to keep (True) or discard (False) proteins in protein_list. Default: True

    Returns
    -------
    pandas Dataframe 
        subsetted dataframe
    '''
    if keep: 
        df_sub = df[list]
        return df_sub
    else: 
        df_sub = df.drop(df.columns[list],axis=1)
        return df_sub

As we want to visualize with Vitessce later, we need to save the created clustering as a separate file. We will split the clustering into three parts: 1) reading in the data, 2) preprocess and cluster, 3) write to files.

In [ ]:
# load in dataframes
def read_dfs_expression_location(uuid, df_expression_file='sprm_outputs/reg001_expr.ome.tiff-cell_channel_total.csv', df_location_file='sprm_outputs/reg001_expr.ome.tiff-cell_centers.csv', root_folder='.'):
    '''
    Read in expression and location dataframes. 

    Parameters
    ----------
    uuid : string
        UUID of dataset
    df_expression_file : string, optional
        relative location of expression csv file.
        Default: 'sprm_outputs/reg001_expr.ome.tiff-cell_channel_total.csv'
    df_location_file : string
        relative location of location csv file.
        Default: 'sprm_outputs/reg001_expr.ome.tiff-cell_centers.csv'
    root_folder : string, optional
        top folder of workspace, or parent folder of datasets. Default: '.'

    Returns
    -------
    Pandas DataFrame 
        dataframe with expression
    Pandas DataFrame
        dataframe with location
    '''
    df_expression = pd.read_csv(root_folder + '/datasets/' + uuid + '/' + df_expression_file)
    df_location = pd.read_csv(root_folder + '/datasets/' + uuid + '/' + df_location_file)
    return [df_expression, df_location]

# preprocess and cluster
def preprocess_and_cluster(df_expression, df_location, n_clusters=10, scaler='z', subset=False, subset_list=[], keep=True):
    '''
    Pre-process and k-means cluster dataset in df_expression. Also updates df_expression and df_location to be consistent.

    Parameters
    ----------
    uuid : string
        UUID of dataset
    df_expression : Pandas DataFrame
        dataframe with expression.
    df_location : Pandas DataFrame
        dataframe with cell locations.
    n_clusters : int, optional
        number of clusters for k-means clustering. Default: 10
    scaler : str, optional
        Scaler used for preprocessing. One of 'z', 'minmax' or None. Default: 'z'
    subset : Boolean, optional
        if true, subset dataframe with proteins in subset_list. Default: False
    subset_list : list of string
        list with proteins to keep. Only used when subset=True. Default: []
    keep : Boolean, optional
        whether to keep (True) or discard (False) proteins in protein_list. 
        Only used when subset=True. Default: True

    Returns
    -------
    Pandas DataFrame 
        dataframe with expression
    Pandas DataFrame
        dataframe with location
    Pandas DataFrame
        dataframe with clusters
    '''
    cellID = list(df_expression['ID'])
    df_location = df_location.loc[df_location['ID'].isin(cellID)]
    if subset: 
        df_expression_subset = protein_subset(df_expression, subset_list)
        df_clusters = kmeans_pipe(df_expression_subset, n_clusters=n_clusters, scaler=scaler)
    else: 
        df_clusters = kmeans_pipe(df_expression, n_clusters=n_clusters, scaler=scaler)
    
    return([df_expression, df_location, df_clusters])

# write to files
def save_dfs(uuid, df_expression, df_location, df_clusters, root_folder='.'):
    '''
    Save dataframes locally.

    Parameters
    ----------
    uuid : string
        UUID of dataset
    df_expression : Pandas DataFrame
        dataframe with expression.
    df_location : Pandas DataFrame
        dataframe with cell locations.
    root_folder : string, optional
        top folder of workspace, or parent folder of datasets. Default: '.'
    '''
    os.makedirs(root_folder + '/output/' + uuid + '/kmeans_output', exist_ok=True)
    df_expression.to_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_channel_total.csv', index=False)
    df_location.to_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_centers.csv', index=False)
    df_clusters.to_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_cluster.csv', index=False)

In [ ]:
# combine reading, clustering, and writing
def get_clusters(uuid, 
                 df_expression_file='sprm_outputs/reg001_expr.ome.tiff-cell_channel_total.csv', 
                 df_location_file='sprm_outputs/reg001_expr.ome.tiff-cell_centers.csv', 
                 root_folder='.',
                 n_clusters=10, scaler='z', subset=False, subset_list=[], keep=True): 
    '''
    Pre-process and k-means cluster dataset identified with UUID, using total protein levels.
    Saves total protein, cell centers and clusters to separate files in kmeans_output folder.

    Parameters
    ----------
    uuid : string
        UUID of dataset
    df_expression_file : string, optional
        relative location of expression csv file.
        Default: 'sprm_outputs/reg001_expr.ome.tiff-cell_channel_total.csv'
    df_location_file : string
        relative location of location csv file.
        Default: 'sprm_outputs/reg001_expr.ome.tiff-cell_centers.csv'
    root_folder : string, optional
        top folder of workspace, or parent folder of datasets. Default: '.'
    n_clusters : int, optional
        number of clusters for k-means clustering. Default: 10
    scaler : str, optional
        Scaler used for preprocessing. One of 'z', 'minmax' or None. Default: 'z'
    subset : Boolean, optional
        if true, subset dataframe with proteins in subset_list. Default: False
    subset_list : list of string
        list with proteins to keep. Only used when subset=True. Default: []
    keep : Boolean, optional
        whether to keep (True) or discard (False) proteins in protein_list. 
        Only used when subset=True. Default: True
    '''
    # load in dataframes
    df_expression, df_location = read_dfs_expression_location(uuid, df_expression_file=df_expression_file, 
                                                              df_location_file=df_location_file, root_folder=root_folder)
    df_expression, df_location, df_clusters = preprocess_and_cluster(df_expression, df_location, n_clusters=n_clusters, 
                                                                     scaler=scaler, subset=subset, subset_list=subset_list, keep=keep)
    save_dfs(uuid, df_expression, df_location, df_clusters, root_folder=root_folder)
    

In [ ]:
for uuid in uuids: 
    get_clusters(uuid)

## Visualize clustering
We can now visualize our new clustering. We will first make a static visualization with matplotlib, and then load the new clustering in Vitessce for an interactive visualization.

In [ ]:
# matplotlib 
def show_kmeans_scatter(uuid, root_folder='.'):
    '''
    Create matplotlib scatterplot of cells, colored by cluster.

    Parameters
    ----------
    uuid : str
        uuid of HuBMAP dataset to visualize.
    '''
    df_location = pd.read_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_centers.csv')
    df_clusters = pd.read_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_cluster.csv')

    df_location['x'] = df_location['x'].map(lambda x: -x)
    col = df_clusters['cluster']
    plt.scatter('y','x',c=col,s=.05,data=df_location,cmap='tab10')
    plt.rcParams['figure.figsize'] = [8,8]
    plt.show()

In [ ]:
show_kmeans_scatter(uuids[0])

# Vitessce view
We can load our data into vitessce for an interactive view.

In the HuBMAP workspaces environment, to load local files, we will first create an AnnDataWrapper for this.

In [ ]:
# Creating the anndata object and writing it to a local zarr store

uuid = uuids[0]

with open(root_folder + '/output/' + uuid + '/kmeans_output/cell_channel_total.csv') as df_expression:
    adata = ad.read_csv(df_expression)

df_expression = pd.read_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_channel_total.csv')
df_location = pd.read_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_centers.csv')
df_clusters = pd.read_csv(root_folder + '/output/' + uuid + '/kmeans_output/cell_cluster.csv')

# df_expression
df_expression.drop('ID', axis=1, inplace=True)
df_location.drop('ID', axis=1, inplace=True)
df_clusters.drop('ID', axis=1, inplace=True)

adata = ad.AnnData(X=df_expression,obs=df_clusters,var=pd.DataFrame(columns=df_expression.columns).T)
adata.obsm['location'] = np.array(df_location[['x', 'y']])

adata.write_zarr(root_folder + '/output/' + uuid + '/anndata.zarr')


In [ ]:
# creating an AnnDataWrapper

w = AnnDataWrapper(
    adata_store=root_folder + '/output/' + uuid + '/anndata.zarr',
    obs_feature_matrix_path="X",
    obs_locations_path="obsm/location",
    obs_set_paths=["obs/cluster"],
    obs_set_names=['K-means Z-scaling k=10'],
    coordination_values={"featureType": "antigen"}
)

In [ ]:
# Creating the Vitessce config

conf = VitessceChainableConfig(
    name="Clustering CODEX data", description="", schema_version="1.0.16"
).add_dataset(
    name='Protein dataset', 
    uid='A',
    objs = [w]
).set_coordination_value(
    c_type="spatialSegmentationLayer",
    c_scope="A",
    c_value={
      "opacity": 1,
      "radius": 20,
      "visible": True,
      "stroked": False
    }
).set_coordination_value(
    c_type="featureType",
    c_scope="A",
    c_value="antigen"
).add_view(
     dataset_uid="A", component="spatial", x=0, y=0, w=4, h=8,
     coordination_scopes={"spatialSegmentationLayer": "A", "featureType": "A" }
).add_view(
    dataset_uid="A", component="obsSetSizes", x=4, y=0, w=4, h=8
).add_view(
    dataset_uid="A", component="obsSets", x=8, y=0, w=2, h=8
).add_view(
    dataset_uid="A", component="heatmap", x=0, y=8, w=6, h=6,
    coordination_scopes={ "featureType": "A" },
    props={"transpose": True},
).add_view(
    dataset_uid="A", component="featureList", x=6, y=8, w=4, h=6,
    coordination_scopes={ "featureType": "A" },
)

conf.widget()

In [ ]:
## If you get a JavaScript error when running the above cell, it is likely due 
## to Anywidget needing to be installed before the workspace is launched.

## Check that anywidget is installed
## Uncomment the following line:
# import anywidget 

## If it is not yet installed:
# !pip install anywidget

## Once you have checked that it is installed, close this window.
## In the Workspace overview page, stop all jobs.
## Then, launch the Workspace again.